## CO2 flux analysis

This script gets the raw inversion data and calculates 

Later (shorter) time series are driven by data from more stations.

### Methods:

1. Load inversion CO2 data and preprocess:
    - select roi
    - adjust units
    - set nodata
    - ...
3. Create a grouping coordinate that divides growing and non-growing seasons.   
    - This can be done manually, but consider fitting a harmonic to divide seasons.  
4. For each year and season, calculate the CO2 flux sum.
5. For each year, get the difference between growing and non-growing season.
    - This is the CO2 flux amplitude that can then be related to atmospheric CO2 and driving variables. 


In [ ]:
# Get all required packages

#%%
%load_ext autoreload
%autoreload 2

# Packages
import os

# Local modules
from mod_ra1 import *

### Options

In [ ]:
# Choose the list of inversion data version to analyse. Later (shorter) time series are driven by data from more stations.
inversions = {'s76':1976, 's81':1981, 's85':1985, 's99':1999, 's06':2006, 's10':2010}  # Inversion starting year. Options: s85, s99, s06, s10
# inversions = {'s76':1976, 's81':1981}  # Inversion starting year. Options: s85, s99, s06, s10

# List of strating years to define analysis periods
periods = [(1976,2021), (1981,2021), (1985,2021), (1999,2021), (2006,2021), (2010,2021)]
# periods = [(2006,2021), (2010,2021)]

### General Settings

In [ ]:
# Directories
input_dir = '../data_input/'
output_dir = '../data_output/'

# Default data properties
epsg_crs = 4326

# Region files available
conts_file = os.path.join(input_dir, 'continents.geojson')
north50_file = os.path.join(input_dir, 'north50.geojson')
north30_file = os.path.join(input_dir, 'north30.geojson')
north0_file = os.path.join(input_dir, 'north0.geojson')
testrect_file = os.path.join(input_dir, 'testrect.geojson')

# Set the region to use for the analysis
roi_file = conts_file

In [ ]:
# Loop over inversion versions and analysis periods

for inv in inversions:

    # Chose the inversion data version. Later (shorter) time series are driven by data from more stations.
    fname = inv + 'oc_v2022_daily.nc'
    invco2_file = '/Users/moyanofe/BigData/GeoSpatial/Carboscope/Inversions/' + fname

    print('Analysing inversion file: ', fname)

    # Call function for inversion CO2 preprocessing
    co2fluxProc = procInvCO2(invco2_file=invco2_file, roi_file=roi_file)

    # Call function to get the yearly seasonal CO2 flux amplitude
    co2fluxAmp = get_CO2fluxSeasAmp(co2fluxProc)

    # Save amps to file
    file_out_amp = os.path.join(output_dir, 'co2fluxAmp_'+inv+'.nc')
    co2fluxAmp.to_netcdf(file_out_amp)
    
    # Loop over periods to get temporal statistics for each:
    
    inv_start = inversions[inv]

    for period in periods:

            print('inv_start: ', inv_start)
            print('period: ', period)
        
            if(inv_start > period[0]):
                print('Period not within inversion run. Skipping ...')
                continue

            print('Executing analysis with: ', inv, ', period: ', period)

            # Call function to calculate temporal trends, mean, variance, etc in seasonal amplitudes
            co2fluxAmpStats = get_co2fluxampstats(co2fluxAmp, period=period)

            # Save amp stats
            ident = inv + '_' + str(period[0])+'-'+str(period[1])
            file_out_stats = os.path.join(output_dir, 'co2fluxAmpStats_'+ident+'.nc')
            co2fluxAmpStats.to_netcdf(file_out_stats)


# -----
# Save the trend data to GeoTiff (useful if reading into Earth Engine is required)
# Saving the database (not as an array) keeps the band name info. (But seems like Earth Engine does not import band names)
# file_out = os.path.join(output_dir, 'co2fluxAmpStats_Inv' + ident + '.tif')
# co2fluxAmpStats.rio.to_raster(file_out)

# Save the co2 flux seasonal difference data to GeoTiff
# file_out = os.path.join(output_dir, 'co2fluxAmp_Inv' + ident + '.tif')
# da_fluxamp = co2fluxAmp['co2flux_yearlyamp']
# da_fluxamp.rio.to_raster(file_out)